### Importing Packages
The following code imports the necessary packages:

In [22]:
# Step 1: Install dependencies (run this in a cell)
# %pip install openai chromadb sentence-transformers

import os
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.utils import embedding_functions
from openai import OpenAI, AsyncOpenAI

from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

In [2]:
# Step 2: Prepare the document chunks (simulate a long forum post)
forum_chunks = [
    {"floor": 1, "user": "楼主", "text": "大家好，我的电脑最近老是自动重启，怎么办？"},
    {"floor": 3, "user": "Alice", "text": "可能是你的电源出了问题，建议检查一下插座。"},
    {"floor": 5, "user": "Bob", "text": "我遇到过类似的问题，结果是 CPU 过热。"},
    {"floor": 8, "user": "Charlie", "text": "你有没有在夏天开空调？我夏天开不了空调，电脑常死机。"}
]

In [ ]:
# Step 3: Embed the documents using a multilingual model
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
texts = [f"{c['user']}({c['floor']}F): {c['text']}" for c in forum_chunks]
embeddings = model.encode(texts)

In [4]:
# Step 4: Create and populate a ChromaDB collection
client = chromadb.Client()
collection = client.create_collection(name="forum")
for idx, (chunk, vector) in enumerate(zip(forum_chunks, embeddings)):
    collection.add(
        documents=[chunk['text']],
        ids=[f"chunk_{idx}"],
        embeddings=[vector],
        metadatas=[{"floor": chunk['floor'], "user": chunk['user']}]
    )

In [41]:
# Step 5: Define a simple query function
def get_completion(prompt, client, model_name, temperature=1.0, max_tokens=1000, top_p=1.0):
    response = client.complete(
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant.",
            },
            {
                "role": "user",
                "content": prompt,
            },
        ],
        model=model_name,
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p
    )
    return response.choices[0].message.content

def query_agent(question: str):
    token = os.environ["GITHUB_TOKEN"]
    endpoint = "https://models.inference.ai.azure.com"
    model_name = "gpt-4o"
    client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)
    question_embedding = model.encode([question])[0]
    results = collection.query(query_embeddings=[question_embedding], n_results=3)
    context = "\n".join([f"{doc['user']}({doc['floor']}F): {doc['text']}" 
                          for doc, meta in zip(forum_chunks, results['metadatas'][0]) 
                          if 'user' in meta])
    prompt = f"""
你是一个懂技术的论坛助手，请根据以下楼层内容回答用户问题。

【帖子内容】
{context}

【用户提问】
{question}

【请简洁回答】
"""
    # Use OpenAI API to get an answer
    response = get_completion(prompt, client, model_name)
    # answer = response.choices[0].message['content']
    print(results['metadatas'][0])
    print("--- Prompt Sent to LLM ---")
    print(prompt)
    print("--- LLM Answer ---")
    print(response)

In [42]:
# Step 6: Run a sample query
query_agent("为什么电脑会突然重启？")

[{'floor': 1, 'user': '楼主'}, {'floor': 5, 'user': 'Bob'}, {'floor': 3, 'user': 'Alice'}]
--- Prompt Sent to LLM ---

你是一个懂技术的论坛助手，请根据以下楼层内容回答用户问题。

【帖子内容】
楼主(1F): 大家好，我的电脑最近老是自动重启，怎么办？
Alice(3F): 可能是你的电源出了问题，建议检查一下插座。
Bob(5F): 我遇到过类似的问题，结果是 CPU 过热。

【用户提问】
为什么电脑会突然重启？

【请简洁回答】

--- LLM Answer ---
电脑突然重启可能是以下原因：  
1. **电源问题**：电源损坏或插座接触不良。  
2. **硬件过热**：如 CPU 或显卡过热会触发自动保护重启。  
3. **驱动或系统问题**：驱动冲突或系统崩溃可能导致重启。  
建议按上述原因逐一检查并排除。


# Behavioral Interview Answer Optimizer

In [ ]:
# Step 1: Install dependencies (already available in GitHub Codespaces)
# If needed: !pip install openai
import os
import openai

from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

In [ ]:
# Step 2: Set up your OpenAI API key (use GitHub Codespaces secret or set manually)
# openai.api_key = os.getenv("OPENAI_API_KEY") or "your-openai-api-key-here"  # Replace as needed

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "gpt-4o"
client = ChatCompletionsClient(endpoint=endpoint, credential=AzureKeyCredential(token))

In [ ]:
# Step 3: Define the prompt template to evaluate behavioral answers
def build_prompt(question, answer):
    return f"""
You are an experienced technical interviewer evaluating a candidate for a senior or staff-level position (L5-L6) in the tech industry. Your task is to analyze the candidate's behavioral interview response with high expectations for leadership, cross-functional coordination, strategic thinking, and measurable impact.

Question:
{question}

Answer:
{answer}

Please evaluate the response in the following aspects:
0. Ensure the time to tell the whole story is 3-5 mins
1. Does the answer follow the STAR format (Situation, Task, Action, Result)?
2. Does the response demonstrate senior-level behaviors such as:
   - Leadership and initiative
   - Influence across teams or functions
   - Strategic problem-solving
   - Clear and measurable outcomes
3. Is the language clear, concise, and persuasive?
4. Provide constructive feedback on how the answer could be improved
5. Rewrite an improved version of the answer, maintaining the original facts but enhancing structure, clarity, and leadership tone

Please:
1. Provide a brief summary in Chinese to call out the pros and cons;
2. Respond in English with a clear, structured and more detailed feedback.

"""

In [18]:
# Step 4: Call OpenAI's ChatCompletion API to analyze and optimize the answer
def evaluate_behavior_answer(question, answer, model="gpt-4o"):
    prompt = build_prompt(question, answer)
    response = client.complete(
        messages=[
            {
                "role": "system",
                "content": "You are a senior technical interviewer for L5-L6 roles in top tech companies. \
                    You evaluate behavioral interview answers with high expectations around leadership, \
                        cross-functional collaboration, and strategic thinking.",
            },
            {
                "role": "user",
                "content": prompt,
            },
        ],
        model=model_name,
        temperature=0.7,
    )
    return response['choices'][0]['message']['content']

In [20]:
# Step 5: Example usage
# question = "Tell me about a time you had to handle a difficult teammate."
# answer = "Once I worked with a teammate who frequently failed to respond to messages, which delayed our project's progress. I proactively scheduled a 1:1 to understand the root cause and found out he was dealing with personal issues. I reorganized some of our responsibilities and increased check-ins, and as a result, we completed the project on time."

target_role = input("Please enter your target interview role (e.g., 'Senior Backend Engineer at Stripe'): ")
question = input("Please enter the interview question: ")
answer = input("Please enter your answer: ")

feedback = evaluate_behavior_answer(question, answer, target_role)
print("\n--- Interviewer Feedback & Suggested Improvements ---\n")
print(feedback)


--- Interviewer Feedback & Suggested Improvements ---

**Summary in Chinese:**

这位候选人的回答触及了解决困难团队成员的问题，但总体表现缺乏高级别职位所预期的领导力、跨团队影响力、战略性问题解决能力以及明确和可量化的结果。优点在于候选人展示了主动性，通过1:1沟通解决问题。然而，回答的细节不足，缺乏对影响范围的清晰说明，也没有展示更广泛的战略思考或显著的业务影响。改进建议包括更明确的STAR结构、更强的领导力语气以及对结果的量化描述。

---

**Detailed Feedback in English:**

1. **Does the answer follow the STAR format?**
   - **Situation:** The candidate briefly describes a teammate failing to respond to messages, causing project delays. However, the situation lacks sufficient context (e.g., what the project was, why timely communication was critical, and the urgency or stakes involved).
   - **Task:** The task is implied as the need to address the teammate’s lack of responsiveness and ensure the project’s timely completion, but it is not explicitly stated.
   - **Action:** The candidate mentions scheduling a 1:1, identifying personal issues, redistributing responsibilities, and increasing check-ins. While these are reasonable actions, they lack strategic dept